In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [26]:
train_data = pd.read_csv('train_data_final.csv')
test_data = pd.read_csv('test_data_final.csv')
X = train_data.drop(['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'], axis=1)
y = train_data['Item_Outlet_Sales']
X_test = test_data.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN input 
X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_val_cnn = X_val_scaled.reshape(X_val_scaled.shape[0], X_val_scaled.shape[1], 1)
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),  # 1D Convolution Layer
    Dropout(0.3),  # Regularization
    Conv1D(filters=32, kernel_size=3, activation='relu'),  # Second Convolution Layer
    Flatten(),  # Flatten the output for Dense layers
    Dense(64, activation='relu'),  # Fully connected layer
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train_cnn, y_train, epochs=100, batch_size=32, validation_data=(X_val_cnn, y_val), verbose=1)
y_pred = model.predict(X_val_cnn)

# Calculate Evaluation Metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)
print("CNN Model Performance on Validation Data:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

Epoch 1/100
214/214 [==============================] - 2s 5ms/step - loss: 4227087.0000 - mae: 1561.8367 - val_loss: 2554965.5000 - val_mae: 1263.6450
Epoch 2/100
214/214 [==============================] - 1s 3ms/step - loss: 2109928.5000 - mae: 1109.8254 - val_loss: 1233108.8750 - val_mae: 806.0084
Epoch 3/100
214/214 [==============================] - 1s 3ms/step - loss: 1413813.8750 - mae: 864.2195 - val_loss: 1124494.7500 - val_mae: 747.9912
Epoch 4/100
214/214 [==============================] - 1s 4ms/step - loss: 1353974.3750 - mae: 828.9927 - val_loss: 1088948.8750 - val_mae: 751.2259
Epoch 5/100
214/214 [==============================] - 1s 4ms/step - loss: 1354997.3750 - mae: 827.4813 - val_loss: 1086783.2500 - val_mae: 735.3260
Epoch 6/100
214/214 [==============================] - 1s 4ms/step - loss: 1347650.8750 - mae: 822.0826 - val_loss: 1084471.7500 - val_mae: 729.7122
Epoch 7/100
214/214 [==============================] - 1s 4ms/step - loss: 1343046.0000 - mae: 818.8751

In [27]:
# Predict on Test Data
predictions = model.predict(X_test_cnn).flatten()
predictions = predictions.flatten()
non_negative_predictions = [max(pred, 0) for pred in predictions]
submission = pd.DataFrame({
    'Item_Identifier': test_data['Item_Identifier'],
    'Outlet_Identifier': test_data['Outlet_Identifier'],
    'Item_Outlet_Sales': predictions  # Now a 1D array
})
print(submission.shape)

178/178 [==============================] - 0s 2ms/step
(5681, 3)


In [28]:
submission.to_csv('dl_model_2.csv', index=False)